In [1]:
import pandas as pd
import numpy as np
import csv
from math import floor
import matplotlib.pyplot as plt
import tensorflow as tf
import onnxruntime as rt
from sklearn.model_selection import train_test_split
import onnx
from onnx.external_data_helper import load_external_data_for_model

In [14]:
DATASET_DIR = '/home/italolanza/workspace/TG/dataset/'
DATASET_NAME = 'dataset_completo.csv' 
TEST_DATA_NAME = 'x_test.pb'
MODEL_DIR = '/home/italolanza/workspace/TG/graduation-project/models/'
MODEL_NAME = 'sequential.onnx'

In [3]:
# feautures
collum_name = ['Tacom_1f0', 'Tacom_2f0', 'Tacom_3f0', 'Tacom_kurtosis', 'Tacom_entropy', 'Aceler_Underhang_X_1f0', 'Aceler_Underhang_X_2f0', 'Aceler_Underhang_X_3f0', 'Aceler_Underhang_X_kurtosis', 'Aceler_Underhang_X_entropy', 'Aceler_Underhang_Y_1f0', 'Aceler_Underhang_Y_2f0', 'Aceler_Underhang_Y_3f0', 'Aceler_Underhang_Y_kurtosis', 'Aceler_Underhang_Y_entropy', 'Aceler_Underhang_Z_1f0', 'Aceler_Underhang_Z_2f0', 'Aceler_Underhang_Z_3f0', 'Aceler_Underhang_Z_kurtosis', 'Aceler_Underhang_Z_entropy', 'Aceler_Overhang_X_1f0', 'Aceler_Overhang_X_2f0', 'Aceler_Overhang_X_3f0', 'Aceler_Overhang_X_kurtosis', 'Aceler_Overhang_X_entropy', 'Aceler_Overhang_Y_1f0', 'Aceler_Overhang_Y_2f0', 'Aceler_Overhang_Y_3f0', 'Aceler_Overhang_Y_kurtosis', 'Aceler_Overhang_Y_entropy', 'Aceler_Overhang_Z_1f0', 'Aceler_Overhang_Z_2f0', 'Aceler_Overhang_Z_3f0', 'Aceler_Overhang_Z_kurtosis', 'Aceler_Overhang_Z_entropy', 'Audio_1f0', 'Audio_2f0', 'Audio_3f0', 'Audio_kurtosis', 'Audio_entropy', 'Class']
all_features = ['Tacom_1f0', 'Tacom_2f0', 'Tacom_3f0', 'Tacom_kurtosis', 'Tacom_entropy', 'Aceler_Underhang_X_1f0', 'Aceler_Underhang_X_2f0', 'Aceler_Underhang_X_3f0', 'Aceler_Underhang_X_kurtosis', 'Aceler_Underhang_X_entropy', 'Aceler_Underhang_Y_1f0', 'Aceler_Underhang_Y_2f0', 'Aceler_Underhang_Y_3f0', 'Aceler_Underhang_Y_kurtosis', 'Aceler_Underhang_Y_entropy', 'Aceler_Underhang_Z_1f0', 'Aceler_Underhang_Z_2f0', 'Aceler_Underhang_Z_3f0', 'Aceler_Underhang_Z_kurtosis', 'Aceler_Underhang_Z_entropy', 'Aceler_Overhang_X_1f0', 'Aceler_Overhang_X_2f0', 'Aceler_Overhang_X_3f0', 'Aceler_Overhang_X_kurtosis', 'Aceler_Overhang_X_entropy', 'Aceler_Overhang_Y_1f0', 'Aceler_Overhang_Y_2f0', 'Aceler_Overhang_Y_3f0', 'Aceler_Overhang_Y_kurtosis', 'Aceler_Overhang_Y_entropy', 'Aceler_Overhang_Z_1f0', 'Aceler_Overhang_Z_2f0', 'Aceler_Overhang_Z_3f0', 'Aceler_Overhang_Z_kurtosis', 'Aceler_Overhang_Z_entropy', 'Audio_1f0', 'Audio_2f0', 'Audio_3f0', 'Audio_kurtosis', 'Audio_entropy']

# loading data
dataset_path = DATASET_DIR + DATASET_NAME

dataset = pd.read_csv(dataset_path, names=collum_name)
dataset_x = dataset[all_features].values
dataset_y = dataset['Class'].values.astype(np.uint8)
x_train, x_test, y_train, y_test = train_test_split(dataset_x, dataset_y, test_size=0.3, shuffle=True)

data_path = DATASET_DIR + TEST_DATA_NAME

# Load a TensorProto
new_tensor = onnx.TensorProto()
with open(data_path, 'rb') as f:
    new_tensor.ParseFromString(f.read())

## Running the ONNX Model

In [15]:
model_path = MODEL_DIR + MODEL_NAME

onnx_model = onnx.load(model_path)
onnx.checker.check_model(onnx_model)

In [36]:

providers = ['CPUExecutionProvider']
sess_options = rt.SessionOptions()
sess_options.enable_profiling = True

session = rt.InferenceSession(model_path, providers=providers, sess_options=sess_options)
input_name = [session.get_inputs()[0].name]
output_names = [session.get_outputs()[0].name]

# onnx_pred = session.run(None, {"input": np.float32(x_test[:3])})


In [37]:
print(input_name)
print(output_names)

['input']
['dense_1']


In [41]:
onnx_pred = session.run(output_names, {"input": np.float32(x_test[:3])})
# onnx_pred = session.run(output_names, {"input": (new_tensor)})

print('ONNX Predicted:', onnx_pred[0])

# make sure ONNX and keras have the same results
# np.testing.assert_allclose(predictions, onnx_pred[0], rtol=1e-5)
print('Model Predicted:', onnx_pred)

ONNX Predicted: [[0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0.]]
Model Predicted: [array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0.]], dtype=float32)]
